In [1]:
from IPython.display import display, HTML
import plotly.graph_objects as go
from plotly import express as px
import pandas as pd
import datetime as dt
import numpy as np
import glob
import os
import re
import warnings

warnings.simplefilter("ignore")

### Goal

Get an overview of Wintershall Noordzee B.V. assets in the North Sea, looking at:
1. Fields and licenses owned by Wintershall since 2015
2. Production of gas, oil and condensates since 2015
3. Share of gas, oil and condensate production in the Dutch North Sea
4. Share of Gazprom

### Data sets

1. License history Netherlands
2. License history with shares UK 2012-2020
3. License history UK
4. Production data Netherlands per field / license

### Steps

1. Get all fields and licenses
2. Compare production data set with historical licenses NL
3. Query UK data for Wintershall Noordzee B.V.
4. Get share of Wintershall in fields, so look for co-use
5. Get share of production in Dutch North Sea production

## Historical licenses

In [2]:
PATH = '/Volumes/GoogleDrive-113330074508532941534/My Drive/Projecten/North Sea data/energy/fossil_fuels/'

In [3]:
# Import historical licenses from Netherlands

df_license = pd.read_excel(PATH + 'licenses/nl_licenses/nl_production_licenses.xlsx', 
                           sheet_name='license_raw', 
                           usecols=['observation_date',
                                    'mutation_start',
                                    'mutation_end',
                                    'mutation_type',
                                    'licensee',
                                    'licensee_normalized',
                                    'license',
                                    'license_normalized',
                                    'nr'])

df_license.sort_values(by='observation_date', ascending=False).head()

,observation_date,mutation_start,mutation_end,mutation_type,nr,licensee,licensee_normalized,license,license_normalized
831,2022-01-01,NaT,NaT,NaN,88,Neptune Energy Participation Netherlands B.V.,Neptune Energy Group,D12a,D12a
1425,2022-01-01,NaT,NaT,NaN,72,TAQA Offshore B.V.,TAQA,P18c,P18c
869,2022-01-01,NaT,NaT,NaN,45,ONE-Dyas B.V.,ONE-Dyas,L11c,L11c
1409,2022-01-01,NaT,NaT,NaN,70,TAQA Offshore B.V.,TAQA,"P15c, P15g, P15h, P15i & P15j","P15c, P15g, P15h, P15i & P15j"
1411,2022-01-01,NaT,NaT,NaN,71,TAQA Offshore B.V.,TAQA,P18a,P18a


In [4]:
def assign_start_date(row):
    if row['mutation_start'] is pd.NaT:
        value = row['observation_date']
    else:
        value = row['mutation_start']
    return value

def assign_end_date(row):
    if row['mutation_end'] is pd.NaT:
        value = row['observation_date'] + pd.offsets.DateOffset(years=1)
    else:
        value = row['mutation_end']
    return value

df_license['start_date'] = df_license.apply(assign_start_date, axis=1)
df_license['end_date'] = df_license.apply(assign_end_date, axis=1)
df_license.head()

,observation_date,mutation_start,mutation_end,mutation_type,nr,licensee,licensee_normalized,license,license_normalized,start_date,end_date
0,2021-01-01,2019-12-20,NaT,Acquired,63,Aceiro Energy B.V.,Aceiro Energy B.V.,P08a,P08a,2019-12-20,2022-01-01
1,2022-01-01,NaT,NaT,NaN,64,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a,P09a,2022-01-01,2023-01-01
2,2017-01-01,NaT,NaT,NaN,65,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a & P09b,P09a,2017-01-01,2018-01-01
3,2016-01-01,NaT,NaT,NaN,65,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a & P09b,P09a,2016-01-01,2017-01-01
4,2015-01-01,NaT,NaT,NaN,59,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a & P09b,P09a,2015-01-01,2016-01-01


In [5]:
# Get min and max observation dates

dfg_license = df_license.groupby(['licensee_normalized', 'license_normalized'])
dfg_license = df_license.agg(start_date=('start_date', np.min), end_date=('end_date', np.max))

# Re-index the dataframe

dfg_license = df_license.reset_index()

# And see the result

dfg_license.head()

,index,observation_date,mutation_start,mutation_end,mutation_type,nr,licensee,licensee_normalized,license,license_normalized,start_date,end_date
0,0,2021-01-01,2019-12-20,NaT,Acquired,63,Aceiro Energy B.V.,Aceiro Energy B.V.,P08a,P08a,2019-12-20,2022-01-01
1,1,2022-01-01,NaT,NaT,NaN,64,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a,P09a,2022-01-01,2023-01-01
2,2,2017-01-01,NaT,NaT,NaN,65,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a & P09b,P09a,2017-01-01,2018-01-01
3,3,2016-01-01,NaT,NaT,NaN,65,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a & P09b,P09a,2016-01-01,2017-01-01
4,4,2015-01-01,NaT,NaT,NaN,59,Aceiro Energy B.V.,Aceiro Energy B.V.,P09a & P09b,P09a,2015-01-01,2016-01-01


## Historical Production data

In [3]:
# Import production license data

dfs= []
for file in glob.glob(PATH + 'production/nl_production/per_license/lice*.xlsx'):
    commodity = os.path.basename(file)[7:-18]
    df = pd.read_excel(file, skiprows=1, engine='openpyxl')
    df['type'] = commodity
    dfs.append(df)
    
df_production = pd.concat(dfs)
len(df_production)

2497

In [4]:
df_production = df_production.drop(['LAND/SEA'], axis=1)

In [6]:
# Melt month columns into one colum

df_production = df_production.melt(id_vars = ['LICENCE', 'OPERATOR', 'YEAR', 'type'], var_name='month', value_name='1000_sm3')

# Clean NaNs
df_production = df_production.dropna(subset=['YEAR'])

# Convert month name and year to date column

df_production['YEAR'] = df_production['YEAR'].astype('int')

df_production['date'] = pd.to_datetime(df_production['YEAR'].astype(str) + '-' + df_production['month'], format='%Y-%b')

# Drop columns and rename

df_production = df_production.rename(columns={'LICENCE': 'license',
                                              'OPERATOR': 'operator_name'})

df_production = df_production.drop(['YEAR', 'month'], axis=1)

df_production['country'] = 'Netherlands'

df_production['year'] = pd.to_datetime(df_production['date']).dt.to_period('Y')

# See the result

len(df_production)

29244

In [7]:
df_production.to_csv(PATH + 'production/production_total.csv')

In [21]:
# Some licenses are missing from the production per license data set, but they are present in the production per field
# data, so let's add those as well.
# The fields are ['A15-A', 'A18-FA', 'B18-FA', 'J03-C', 'L08-A', 'N07-FA']

fields = ['A15-A', 'A18-FA', 'B18-FA', 'J03-C', 'L08-A', 'N07-FA']

dfs= []
for file in glob.glob(PATH + 'production/nl_production/per_field/field*.xlsx'):
    commodity = os.path.basename(file)[5:-18]
    df = pd.read_excel(file, skiprows=1, engine='openpyxl')
    df['type'] = commodity
    df = df[df['FIELD'].isin(fields)]
    dfs.append(df)
    
df_fields = pd.concat(dfs)

len(df_fields)

53

In [ ]:
# Melt month columns into one colum

df_fields = df_fields.melt(id_vars = ['FIELD', 'OPERATOR', 'YEAR', 'type'], var_name='month', value_name='1000_sm3')

# Clean NaNs
df_fields = df_fields.dropna(subset=['YEAR'])

# Convert month name and year to date column

df_fields['YEAR'] = df_fields['YEAR'].astype('int')

df_fields['date'] = pd.to_datetime(df_fields['YEAR'].astype(str) + '-' + df_fields['month'], format='%Y-%b')

# Drop columns and rename

df_fields = df_fields.rename(columns={'FIELD': 'license',
                                      'OPERATOR': 'operator_name'})

df_fields = df_fields.drop(['YEAR', 'month'], axis=1)

df_fields['country'] = 'Netherlands'

df_fields['year'] = pd.to_datetime(df_fields['date']).dt.to_period('Y')

# See the result

len(df_fields)

In [ ]:
df_fields.head()

In [ ]:
# Bring it all together

df_production = pd.concat([df_production, df_fields])
len(df_production)

## Try filtering out Wintershall

In [22]:
com = list(set(df_license[df_license['licensee_normalized'].str.contains('Gaz')]['license_normalized']))

In [23]:
com.sort()
com

['D12a',
 'D12b',
 'D15a',
 'D18a',
 'E15a',
 'E15b',
 'E18a',
 'F13a',
 'F16a & F16b',
 'F17a-diep',
 'G17c & G17d',
 'K08 & K11a',
 'K18b',
 'L05b',
 'L05c',
 'L06a',
 'L06b',
 'L08-A',
 'L08b, L08d & L08e',
 'L12a',
 'L12b & L15b',
 'L12b & L15d',
 'L12c',
 'L12d',
 'L13',
 'L15d',
 'L16a',
 'P06a',
 'P09a',
 'P09a, P09b & P09d',
 'P09c',
 'P12a',
 'P15a, P15b, P15d, P15e & P15f',
 'P15c, P15g, P15h, P15i & P15j',
 'Q01c-diep',
 'Q04a',
 'Q05d']

In [24]:
df_production = df_production[df_production['date'] >= '2015-01-01']
com_prod = df_production[df_production['license'].isin(com)]
len(com_prod)

3684

In [27]:
com_prod.head()

,license,operator_name,type,1000_sm3,date,country,year
3,D12a,Wintershall Noordzee B.V.,Gas,98753.074310,2021-01-01,Netherlands,2021
4,D15a,Neptune Energy Netherlands B.V.,Gas,0.000000,2021-01-01,Netherlands,2021
5,D18a,Neptune Energy Netherlands B.V.,Gas,0.000000,2021-01-01,Netherlands,2021
11,F16a & F16b,Wintershall Noordzee B.V.,Gas,0.000000,2021-01-01,Netherlands,2021
14,G17c & G17d,Neptune Energy Netherlands B.V.,Gas,7510.426201,2021-01-01,Netherlands,2021


In [28]:
com_prod = com_prod[com_prod['type'] == 'Gas']

prod_win = pd.DataFrame(com_prod.groupby('license')['1000_sm3'].sum())

In [29]:
prod_win['1000_sm3'] = prod_win['1000_sm3'] / 1000000

In [30]:
prod_win

,1000_sm3
license,
D12a,1.879926
D15a,0.022851
D18a,0.158088
E18a,0.164300
F16a & F16b,0.688225
G17c & G17d,0.734572
K08 & K11a,2.283953
K18b,2.449609
L05b,0.885985


In [ ]:
prod_win.to_csv(PATH + 'licenses/nl_licenses/production_wintershall.csv')

In [31]:
df = pd.merge(pd.DataFrame(com_prod.groupby('year')['1000_sm3'].sum()), 
         pd.DataFrame(df_production.groupby('year')['1000_sm3'].sum()),
         on='year',
         how='left')



df['perc'] = df['1000_sm3_x'] / df['1000_sm3_y'] * 100

df['1000_sm3_x'] = df['1000_sm3_x'] / 1000000
df['1000_sm3_y'] = df['1000_sm3_y'] / 1000000

df = df.rename(columns={'1000_sm3_x': 'prod_wintershall_1b_sm3',
                        '1000_sm3_y': 'prod_total_1b_sm3',
                        'perc': 'percentage_wintershall'
                       })

df.round({'prod_wintershall_1b_sm3': 2,
          'prod_total_1b_sm3': 2,
          'percentage_wintershall':2})

,prod_wintershall_1b_sm3,prod_total_1b_sm3,percentage_wintershall
year,,,
2015,3.04,16.33,18.63
2016,2.77,15.19,18.25
2017,2.53,13.72,18.48
2018,2.45,12.24,19.99
2019,2.18,10.89,19.99
2020,2.40,10.39,23.08
2021,2.29,9.83,23.32
2022,0.87,10.00,8.65


In [ ]:
df

In [ ]:
# Add other companies

df = pd.merge(pd.DataFrame(com_prod.groupby('year')['1000_sm3'].sum()),
              df,
              on='year',
              how='left')
df

In [ ]:
df['1000_sm3'] = df['1000_sm3'] / 1000000

df['perc'] = df['1000_sm3'] / df['prod_total_1b_sm3'] * 100


df = df.rename(columns={'1000_sm3': 'prod_totalenergies_1b_sm3',
                        '1000_sm3_y': 'prod_nam_sm3',
                        'perc': 'percentage_totalenergies'
                       })

df.round({'prod_totalenergies_1b_sm3': 2,
          'prod_total_1b_sm3': 2,
          'percentage_totalenergies':2})

## Infrastructure

In [ ]:
# Import platforms

df_infra = pd.read_excel(PATH + 'licenses/nl_licenses/nl_production_licenses.xlsx', 
                         sheet_name='platforms')

df_infra.head()

In [ ]:
df_infra = df_infra.dropna(subset=['operator'])

In [ ]:
winfra = df_infra[df_infra['operator'].str.contains('Wintersh')]

In [ ]:
winfra[winfra['year_end'].isna()]['platform'].nunique()

In [ ]:
df_pijpleidingen = pd.read_excel(PATH + 'licenses/nl_licenses/nl_production_licenses.xlsx',
                                sheet_name='pipelines')

df_pijpleidingen.head()

In [ ]:
df_pijpleidingen = df_pijpleidingen.dropna(subset=['operator'])
wpijp = df_pijpleidingen[df_pijpleidingen['operator'].str.contains('Wint')]
wpijp.head()

In [ ]:
wpijp = wpijp.drop_duplicates(subset=['from'])
wpijp['to'].unique()

## All companies

So now compare these dataframes:
1. df_license (historical licenses)
2. nl_production_licenses (license production)

In [ ]:
merge = pd.merge(df_license, 
                 df_production, 
                 left_on='license_normalized', 
                 right_on='license',
                 how='outer')

merge.head()

In [ ]:
merge = pd.merge(df_license,
                df_production,
                 how='outer',
                 left_on='license_normalized',
                 right_on='license'
                )

len(merge)

In [ ]:
# Filter on start_date, end_date and license_name

merge = merge.query('start_date <= date and end_date >= date and license_y == license_normalized' )
len(merge)

In [ ]:
# first explore the differences between historical licenses and production licenses

diff = list(set(dfg_license.license) - set(merge.license_normalized))
diff.sort()
diff

In [ ]:
# Clean it up

# Select relevant columns

df_total = merge[['license_normalized', 
                  'operator_name', 
                  'type', '1000_sm3', 
                  'date', 
                  'country',
                  'licensee',
                  'licensee_normalized', 
                  'start_date', 
                  'end_date', 
                  'year']]

# Rename columns

df_total = df_total.rename(columns={'license_normalized': 'license', 
                                    'date': 'production_date', 
                                    'start_date': 'license_start_date', 
                                    'end_date': 'license_end_date'})

# Filter date range

df_total = df_total[df_total['production_date'] >= '2015-01-01']

df_total.head()

In [ ]:
# Write to file (be aware of date filter!)

df_total.to_csv(PATH + 'licenses/nl_licenses/matched_production.csv', index=False)

# Write total production df to file

df_production.to_csv(PATH + 'licenses/nl_licenses/nl_production_licenses.csv', index=False)

## Analysis

### Datasets

1. df_license: historical licenses EEZ NL from 2015-01-01 onwards
2. dfg_license: grouped by license and licensee (normalized)
3. df_total: production grouped by company
3. df_production: historical production EEZ NL from 2003-01-01 onwards

### Questions

- How much gas is yearly produced since 2015?
- How much gas is yearly produced by each company?
- What are the main trends? 
- How does the gas production of 2022 compare to previous years?
- What is the relative market share of each company (ignoring license shares)?
- What infrastructure do these companies have? 
- Where is this infrastructure located? 
- Which infrastructure can we assume to be decommissioned in the near future (yes, it's fuzzy)?

In [ ]:
# Production per year - table

yearly_production = df_production.groupby('year', as_index=False)['1000_sm3'].sum()
yearly_production['1b_sm3'] = yearly_production['1000_sm3'] / 1000000
yearly_production

In [ ]:
# Production per year - graph
yearly_production = df_production.groupby('date', as_index=False)['1000_sm3'].sum()
yearly_production['1b_sm3'] = yearly_production['1000_sm3'] / 1000000
fig = px.line(yearly_production, x='date', y='1b_sm3')
fig.show()

In [ ]:
# Production per company (not normalized)

production_company = pd.pivot_table(df_total, index=['license', 'year', 'licensee'], values='1000_sm3', aggfunc='sum')
production_company = production_company.reset_index()
production_company.head()

In [ ]:
production_company.columns

In [ ]:
test = production_company.groupby(['license','year', '1000_sm3'])['licensee'].count()
test = pd.DataFrame(test).reset_index()
test.head()

In [ ]:
test['prod_1000_sm3'] = test['1000_sm3'] / test['licensee']
len(test)

In [ ]:
test.head()

In [ ]:
#df_total['year'] = df_total['year'].astype('int64')
#test['year'] = test['year'].astype('int64')

production_company = pd.merge(df_total[['license', 'year', 'licensee']],
                             test[['license', 'year', 'prod_1000_sm3']],
                              on=['license', 'year'],
                              how='left')
len(production_company)

In [ ]:
production_company.head()

In [ ]:
production_company = production_company.drop_duplicates(subset=['license', 'year', 'licensee'])

In [ ]:
production_company.head(20)

In [ ]:
p = production_company.groupby(['licensee', 'year'])['prod_1000_sm3'].sum()
p = p.reset_index()
fig = px.bar(p,
              x=p['year'].dt.strftime('%Y'),
              y='prod_1000_sm3',
              color='licensee')
fig.show()

In [ ]:
p.head(20)

In [ ]:
## Flotsam and jetsam

# License names are different when there is only one digit, e.g. P9 in historical license set is P09 in production license set. Modify by regex in historical license

def pad_license(d):
    digit = re.match('[A-Z](\d{1,2})*', d).group(1)
    result = d.replace(digit, digit.zfill(2))
    return result 

df_license['license'] = df_license['license'].apply(lambda x: pad_license(x))